In [1]:
import numpy as np
import pandas as pd

def get_infos():
    ## 0: Code , 1:Weight , 2: height, 3:age, 4: gender
    dss = np.genfromtxt("data_subjects_info.csv",delimiter = ',')
    dss = dss[1:]
    return dss

def create_time_series(num_features,num_act_labels,num_gen_labels,label_codes,trial_codes):
    dataset_columns = num_features + num_act_labels + num_gen_labels
    ds_list = get_infos()
    train_data = np.zeros((0,dataset_columns))
    test_data = np.zeros((0,dataset_columns))
    for i, sub_id in enumerate(ds_list[:,0]):
        for j, act in enumerate(label_codes):
            for trial in trial_codes[act]:
                fname = 'A_DeviceMotion_data/'+act+'_'+str(trial)+'/sub_'+str(int(sub_id))+'.csv'
                raw_data=pd.read_csv(fname,index_col=0)
                raw_data.drop(raw_data.columns[raw_data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
                unlabel_data=raw_data.values
                label_data=np.zeros((len(unlabel_data),dataset_columns))
                label_data[:,:-(num_act_labels + num_gen_labels)] = unlabel_data
                label_data[:,label_codes[act]] = 1
                label_data[:,-(num_gen_labels)] = int(ds_list[i,4])
                
                if trial>10:
                    test_data=np.append(test_data,label_data,axis=0)
                else:
                    train_data=np.append(train_data,label_data,axis=0)
    
    return train_data,test_data

In [2]:
#labeled time_series from dataset of "(A)DeviceMotion_data"

num_features=12
num_act_labels=4 #dws,ups,wlk,jog
num_gen_labels = 1 # 0(female)/1(male)
label_codes={"dws":num_features, "ups":num_features+1, "wlk":num_features+2, "jog":num_features+3}
trial_codes = {"dws":[1,2,11], "ups":[3,4,12], "wlk":[7,8,15], "jog":[9,16]} 

train_ts,test_ts=create_time_series(num_features, num_act_labels, num_gen_labels, label_codes, trial_codes)
print("Shape of Training Time-Seires:", train_ts.shape)
print("Shape of Test Time-Series:", test_ts.shape)

Shape of Training Time-Seires: (621973, 17)
Shape of Test Time-Series: (145687, 17)


In [3]:
def time_series_to_section(dataset, num_act_labels, num_gen_labels, sliding_window_size, step_size_of_sliding_window, standardize = False, **options):
    data = dataset[: , 0:-(num_act_labels+num_gen_labels)]
    act_labels = dataset[: , -(num_act_labels+num_gen_labels):-(num_gen_labels)]
    gen_labels = dataset[: , -(num_gen_labels)]
    mean = 0
    std = 1
    
    if standardize:
        ## Standardize each sensor’s data to have a zero mean and unity standard deviation.
        ## As usual, we normalize test dataset by training dataset's parameters 
        if options:
            mean = options.get("mean")
            std = options.get("std")
        else:
            mean = data.mean(axis=0)
            std = data.std(axis=0)
            print("Training Data has been standardized:\n the mean is = ",str(mean.mean())," ; and the std is = ",str(std.mean()))            
  
        data -= mean
        data /= std
    else:
        print("----> Without Standardization.....")

    ## We want the Rows of matrices show each Feature and the Columns show time points.
    data = data.T
            
    size_features = data.shape[0]
    size_data = data.shape[1]
    number_of_secs = round(((size_data - sliding_window_size)/step_size_of_sliding_window))
            
    ##  Create a 3D matrix for Storing Snapshots  
    secs_data = np.zeros((number_of_secs , size_features , sliding_window_size ))
    act_secs_labels = np.zeros((number_of_secs, num_act_labels))
    gen_secs_labels = np.zeros(number_of_secs)
    
    k=0    
    for i in range(0 ,(size_data)-sliding_window_size  , step_size_of_sliding_window):
        j = i // step_size_of_sliding_window
        if(j>=number_of_secs):
            break
        if(gen_labels[i] != gen_labels[i+sliding_window_size-1]): 
            continue
        if(not (act_labels[i] == act_labels[i+sliding_window_size-1]).all()): 
            continue    
        secs_data[k] = data[0:size_features, i:i+sliding_window_size]
        act_secs_labels[k] = act_labels[i].astype(int)
        gen_secs_labels[k] = gen_labels[i].astype(int)
        k = k+1
    secs_data = secs_data[0:k]
    act_secs_labels = act_secs_labels[0:k]
    gen_secs_labels = gen_secs_labels[0:k]
    
    return secs_data, act_secs_labels, gen_secs_labels, mean, std
##________________________________________________________________


## This Variable Defines the Size of Sliding Window
## ( e.g. 100 means in each snapshot we just consider 100 consecutive observations of each sensor) 
sliding_window_size = 50 # 50 Equals to 1 second for MotionSense Dataset (it is on 50Hz samplig rate)
## Here We Choose Step Size for Building Diffrent Snapshots from Time-Series Data
## ( smaller step size will increase the amount of the instances and higher computational cost may be incurred )
step_size_of_sliding_window = 10 
print("Sectioning Training and Test datasets: shape of each section will be: (",num_features,"x",sliding_window_size,")")
train_data, act_train_labels, gen_train_labels, train_mean, train_std = time_series_to_section(train_ts.copy(),
                                                                                               num_act_labels,
                                                                                               num_gen_labels,
                                                                                               sliding_window_size,
                                                                                               step_size_of_sliding_window,
                                                                                               standardize = True)

test_data, act_test_labels, gen_test_labels, test_mean, test_std = time_series_to_section(test_ts.copy(),
                                                                                          num_act_labels,
                                                                                          num_gen_labels,
                                                                                          sliding_window_size,
                                                                                          step_size_of_sliding_window,
                                                                                          standardize = True,
                                                                                          mean = train_mean, 
                                                                                          std = train_std)
print(" Shape of Training Sections:", train_data.shape)
print(" Shape of Test Sections:", test_data.shape)

Sectioning Training and Test datasets: shape of each section will be: ( 12 x 50 )
Training Data has been standardized:
 the mean is =  -0.01695311674756102  ; and the std is =  0.8714768261722013
 Shape of Training Sections: (61728, 12, 50)
 Shape of Test Sections: (14098, 12, 50)


In [4]:
from torch.nn import Sequential
import torchvision.models as models
import torch

In [5]:
train_data = np.expand_dims(train_data,axis=3)
test_data = np.expand_dims(test_data,axis=3)
print(" Shape of Training Sections:", train_data.shape)
print(" Shape of Test Sections:", test_data.shape)

 Shape of Training Sections: (61728, 12, 50, 1)
 Shape of Test Sections: (14098, 12, 50, 1)


In [6]:
num_train,height,width,channel = train_data.shape
metrics = ['acc']
conv_depth_1 = 50
kernel_size_1=5
kernel_size_2=3
class CNN(nn.Module):
    
    def __init__(self, ):
        super(CNN, self).__init__()
        self.conv_0 = nn.Conv2d(in_channels=height,out_channels=conv_depth_1,kernel_size=(1,kernel_size_1),padding='valid')
        self.conv_1 = nn.Conv2d(in_channels=conv_depth_1,out_channels=convo_depth_1,kernel_size=(1,kernel_size_2),padding='same')
        self.linear_1=nn.linear(

SyntaxError: unexpected EOF while parsing (<ipython-input-6-862bf8aec744>, line 12)

In [45]:
x = torch.randn((1,3,2,2))

In [47]:
train_data.shape

(61728, 12, 50, 1)

In [52]:
num_train,height,width,channel = train_data.shape
print(height)
print(width)

12
50
